In [42]:
import os
os.listdir('.')

['.config',
 'TASS2017_T1_development.xml',
 'TASS2017_T1_training.xml',
 'test_labels.txt',
 'ElhPolar_esV1.lex',
 'TASS2017_T1_test.xml',
 'sample_data']

In [43]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Exercice 2. Sentiment Analysis

## Modelo 1

In [44]:
from sklearn import svm
from sklearn.metrics import classification_report
from nltk.tokenize import TweetTokenizer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
import re
from sklearn import svm
from sklearn.metrics import classification_report
import scipy
import re
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


train_path = "TASS2017_T1_training.xml"
dev_path = "TASS2017_T1_development.xml"
test_path = "TASS2017_T1_test.xml"

def preprocess_xml(path, is_test=False):
    # Loading data from xml and tokenizing
    x=[]
    y=[]
    ids=[]
    with open(path, "r") as f:
        soup = BeautifulSoup(f, "xml")
    tokenizer = TweetTokenizer(strip_handles=False, reduce_len=True, preserve_case=False)

    for tweet in soup.find_all("tweet"):
        content = tweet.content.string
        x.append(" ".join(tokenizer.tokenize(content)))
        ids.append(tweet.tweetid.string)

        if not is_test:
          sentiment = tweet.sentiment.polarity.value.string
          y.append(sentiment)

    return x,y, ids


def mi_tokenizador(s):
    expresion = ['([0-9]+)(\sde\s)(\w+)(\sde\s)([0-9]+)', '(http(s)*://)([^\s]*)', '([0-9]{1,2})(\/|-|:)([0-9]{1,2})(\/|-|:)([0-9]{4}|[0-9]{2})',
                  '([^\s]+@[^\s]+)', '([@|#]([^\s]+))', '([À-úA-Z]+\.)+', '([0-9])+(\.|,|\-|\:|\/)([0-9]+)',  '([À-úa-zA-Z])+(\-)([À-úa-zA-Z]+)',
                  '([^\w\s.:;,!¡/@¿?%~\"\'\#-])', '\(', '\)', '\.\.\.', '\.', '\,', '\'', '\"', '\?', '\¿', '\!','\¡', '\;', '\:','\%','\w+']
    reg_exp = "|".join(expresion)
    compilador= re.compile(reg_exp, re.U)
    return [i.group(0)  for i in re.finditer(compilador, s)]


# Data preprocessing
train_x, train_y, _ = preprocess_xml(train_path)
dev_x, dev_y, _ = preprocess_xml(dev_path)
test_x, test_y, test_ids = preprocess_xml(test_path, is_test=True)

# Vectorize the data
vectorizer = CountVectorizer(tokenizer=mi_tokenizador)
train_vectors = vectorizer.fit_transform(train_x)
dev_vectors = vectorizer.transform(dev_x)

# Train classifier
classifier_liblinear = svm.LinearSVC(C=0.1)
classifier_liblinear.fit(train_vectors, train_y)

# Eval classifier with dev
prediction_liblinear = classifier_liblinear.predict(dev_vectors)
print(classification_report(dev_y, prediction_liblinear))

              precision    recall  f1-score   support

           N       0.60      0.79      0.68       219
         NEU       0.27      0.10      0.15        69
        NONE       0.26      0.11      0.16        62
           P       0.60      0.62      0.61       156

    accuracy                           0.56       506
   macro avg       0.43      0.41      0.40       506
weighted avg       0.51      0.56      0.52       506



In [45]:
test_vectors = vectorizer.transform(test_x)
predicted_labels = classifier_liblinear.predict(test_vectors)
with open("test_labels.txt", 'w') as f:
  for i in range(len(test_ids)):
    id = test_ids[i]
    text = test_x[i]
    pred = predicted_labels[i]
    f.write(id+'\t'+pred+'\n')